# Some bigdata examples with Kita Lab/Bioconductor

Bioconductor 3.8 includes facilities for interfacing with large data through
SummarizedExperiments using HDF Scalable Data Service to store and query assay quantifications.

# The 10x 1.3 million neuron dataset

## The annotation shell in a SummarizedExperiment

We start with an archived SummarizedExperiment that lacks assay data.  We placed this in Bioconductors'
centralized ExperimentHub.

In [ ]:
ehub = ExperimentHub::ExperimentHub()
tag = names(AnnotationHub::query(ehub, "full_1Mneurons"))
tenx = ehub[[tag[1]]]

In [ ]:
tenx

## A DelayedArray instance for the matrix of counts

We need the very latest rhdf5client package.

In [ ]:
ds = rhdf5client::H5S_Array(endpoint = rhdf5client::URL_hsds(), 
            filepath = "/shared/bioconductor/tenx_full.h5", host = "newassay001")
ds

## Bind the assay quantifications to the SummarizedExperiment

In [ ]:
SummarizedExperiment::assays(tenx) = S4Vectors::SimpleList(counts=ds)

In [ ]:
assay(tenx)

Note that the row and column names are available after this assignment.

## Querying a gene set

Tasic et al. (Nature Neuroscience 19(2):335-346, 2016) define modules of the mouse visual cortex.
One group of genes, cluster f01, is defined in Supplementary table S8.  We'll assemble summary statistics on these genes in the first  10000 cells in the array.

In [ ]:
tasic1 = c("Crispld2", "Cxcl14", "Tpm2", "Itih5", "Cox6a2")

In [ ]:
selTenx = tenx[which(rowData(tenx)$symbol %in% tasic1),]
selTenx

In [ ]:
apply(assay(selTenx[,1:10000]),1,function(x)summary(as.numeric(x)))

# A compendium of (most) human transcriptomes archived in NCBI SRA

## A view of metadata

We'll use the SRAdbV2 package of Sean Davis to get access to metadata and
accession numbers for samples in NCBI SRA.  See https://api-omicidx.cancerdatasci.org/sra/1.0/ui/

In [ ]:
library(devtools)
install_github("seandavi/SRAdbV2")

In [ ]:
library(SRAdbV2)

In [ ]:
idx = Omicidx$new()
sr = idx$search(q = "sample_taxon_id: 9606 AND experiment_library_source: transcriptomic")
sr$count()

In [ ]:
scr1 = sr$scroll()
lk1 = scr1$yield()

In [ ]:
head(unique(lk1$study_title))

## Selecting metadata on a study of interest

We'll obtain metadata on the N=85 (79+7-1, 1 ad hoc missing record) in the fourth study above,
which corresponds to PMID 27093186.

In [ ]:
sr2 = idx$search(q = 'study_title: "small cell lung cancer (sclc)"')

In [ ]:
sr2$count()

In [ ]:
sclcMeta = as.data.frame(sr2$scroll()$collate()) # not tibble

In [ ]:
dim(sclcMeta)

## Acquiring the human transcriptome compendium RESTful SummarizedExperiment

Sean Davis of NCI has created bigrna.cancerdatasci.org to archive salmon quantifications
of all human RNA-seq data in NCBI SRA.  We have harvested this using Bioconductor tximport to obtain
the gene-level quantifications for 181134 RNA-seq studies.  The numerical data are in HSDS and
can be accessed using the htxcomp package as noted here.

In [ ]:
library(devtools)
Sys.setenv(TAR="/bin/tar")
install_github("vjcitn/htxcomp")

In [ ]:
library(htxcomp)

In [ ]:
htx = loadHtxcomp()

In [ ]:
htx

We use standard syntax for SummarizedExperiments to (lazily) confine the assay
and metadata to those in the SCLC experiments

In [ ]:
sclcES = htx[, intersect(sclcMeta$experiment_accession, colnames(htx))] # one missing!
sclcES

In [ ]:
assay(sclcES[1:4,1:4])

Here we build up the 'colData' component of the SummarizedExperiment.

In [ ]:
rownames(sclcMeta) = sclcMeta$experiment_accession

In [ ]:
colData(sclcES) = S4Vectors::DataFrame(sclcMeta[colnames(sclcES),])

In [ ]:
sclcES

`sclcMeta` has a complex structure, it is not a flat table.  We extract sample type
information using sapply.

In [ ]:
sclcES$samptype = sapply(colData(sclcES)$sample_attributes, function(x)x[1,2])

In [ ]:
table(sclcES$samptype)

Finally, we find the row for gene SRSF1 and compare its expression between normal and tumor samples.

In [ ]:
grep("ENSG00000136450", rownames(sclcES)) # SRSF1

In [ ]:
boxplot(split(as.numeric(assay(sclcES[7284,])), sclcES$samptype))